In [1]:
from datetime import datetime, timedelta
import math

from bokeh.io import output_notebook
from bokeh.models import FuncTickFormatter, ColumnDataSource
from bokeh.plotting import figure, output_file, show
import pandas as pd
import numpy as np

# bokeh: configure for notebook
# https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html#userguide-jupyter-notebook
output_notebook()

# load data
df = pd.read_csv(
    "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
)
df.date = df.date.apply(pd.to_datetime)
df.head()

Loading BokehJS ...

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [2]:
def get_r_squared(x, y, coeffs):
    # source: https://stackoverflow.com/questions/893657/
    p = np.poly1d(coeffs, r=True)
    # fit values, and mean
    yhat = p(x)  # or [p(z) for z in x]
    ybar = np.sum(y) / len(y)  # or sum(y)/len(y)
    ssreg = np.sum(
        (yhat - ybar) ** 2
    )  # or sum([ (yihat - ybar)**2 for yihat in yhat])
    sstot = np.sum((y - ybar) ** 2)  # or sum([ (yi - ybar)**2 for yi in y])
    return 1 - (ssreg / sstot)

def unbiased_polyfit(df, steps=10):
    # source: https://stackoverflow.com/questions/3433486
    # generate x y values for calculation
    x, y = list(range(len(df.index))), df[("cases", "sum")]
    b, a_ = np.polyfit(x, np.log(y), 1, w=np.sqrt(y))
    a = math.exp(a_)
    r_squared = get_r_squared(x, y, [b, a_])
    print(f"y ≈ exp({a_}) * exp({b} * x)")
    print(f"y = {a} * exp({b} * x)")
    # print(f'r^2 = {r_squared}')
    fnc = lambda x: a * math.exp(b * x)
    x_range = [i for i in range(len(df.index) + steps)]
    line = [fnc(x_val) for x_val in x_range]
    return line


def make_figure(df, title):
    # make the figure
    p = figure(
        title=title,
        x_axis_label="Date",
        y_axis_label="# of Cases & Deaths",
        y_range=[0, int(df[("cases", "sum")].max() * 1.20)],
        x_range=[df.index[0], pd.Timestamp(df.index.max() + timedelta(days=2))],
        plot_width=880,
        tools="pan,wheel_zoom,box_zoom,reset",
    )
    # add actual values as an area graph
    source = ColumnDataSource(
        data=dict(x=df.index, cases=df[("cases", "sum")], deaths=df[("deaths", "sum")],)
    )
    p.varea_stack(["deaths", "cases"], x="x", color=("red", "lightblue"), source=source)
    # add unbiased polyfit
    steps = 10
    extended_x = list(df.index) + [
        pd.Timestamp(df.index.max() + timedelta(days=i)) for i in range(1, steps + 1)
    ]
    p.line(
        extended_x,
        unbiased_polyfit(df, steps=steps),
        legend_label=f"Projected # of Cases",
        line_width=1,
        line_color="gray",
        line_dash="dashed",
        line_alpha=0.75,
    )
    # format axis
    label_dict = {str(d): d.strftime("%d-%m-%Y") for d in df.index}
    p.xaxis.formatter = FuncTickFormatter(
        code="""
        let date = new Date(tick);
        return `${date.getDate()}-${date.getMonth()}-${date.getFullYear()}`
    """
    )
    p.legend.location = "top_left"
    show(p)


def plot_state_curve(df, state: str):
    # get the state data
    state_df = df[df["state"] == state]
    # arrange by day
    day_df = (
        state_df.drop(columns=["fips", "state", "county"])
        .groupby(by="date")
        .agg(["sum"])
    )
    day_df.columns = {("cases", "sum"): "cases_sum", ("deaths", "sum"): "deaths_sum"}
    make_figure(day_df, f"{state} Cases")


def plot_state_counties_curve(df, state: str, counties: list):
    # get the state and county data
    counties_df = df[(df.state == state) & (df.county.isin(counties))]
    # arrange by day
    day_df = (
        counties_df.drop(columns=["fips", "state", "county"])
        .groupby(by="date")
        .agg(["sum"])
    )
    day_df.columns = {("cases", "sum"): "cases_sum", ("deaths", "sum"): "deaths_sum"}
    make_figure(day_df, f'{state} Cases for counties {",".join(counties)}')


def plot_states_and_counties_curve(df, state_county_map: dict, name):
    """for metropolitan area. i.e. chicagoland"""
    states = state_county_map.keys()
    # trim dataset to relevant states, counties
    state_counties_df = df[
        df.apply(
            lambda x: x["state"] in states
            and x["county"] in state_county_map[x["state"]],
            axis=1,
        )
    ]
    # arrange by day
    day_df = (
        state_counties_df.drop(columns=["fips", "state", "county"])
        .groupby(by="date")
        .agg(["sum"])
    )
    day_df.columns = {("cases", "sum"): "cases_sum", ("deaths", "sum"): "deaths_sum"}
    make_figure(day_df, name)

In [3]:
plot_state_curve(df, "Indiana")

y ≈ exp(5.412883621451692) * exp(0.09263487302211054 * x)
y = 224.27738719722757 * exp(0.09263487302211054 * x)


In [4]:
plot_state_counties_curve(df, "Indiana", ['Lake'])

y ≈ exp(3.6018835384247403) * exp(0.10866217631061334 * x)
y = 36.66723358539854 * exp(0.10866217631061334 * x)


In [5]:
plot_state_counties_curve(df, "Indiana", ['Marion'])

y ≈ exp(4.811847112354874) * exp(0.08210229410329578 * x)
y = 122.95852610262638 * exp(0.08210229410329578 * x)


In [6]:
plot_state_counties_curve(df, 'Illinois', ['Cook'])

y ≈ exp(1.9506576278537757) * exp(0.09512395614646768 * x)
y = 7.0333113615147 * exp(0.09512395614646768 * x)


In [7]:
# https://en.wikipedia.org/wiki/Chicago_metropolitan_area
chicago_metropolitan = dict(
    Illinois=[
        "Cook",
        "DeKalb",
        "DuPage",
        "Grundy",
        "Kankakee",
        "Kane",
        "Kendall",
        "McHenry",
        "Will",
    ],
    Indiana=["Jasper", "Lake", "Newton", "Porter",],
    Wisconsin=["Kenosha"],
)

plot_states_and_counties_curve(
    df, chicago_metropolitan, "Cases for the Chicago Metropolitan Area"
)

y ≈ exp(2.130988819925073) * exp(0.09613020351321211 * x)
y = 8.423191714646883 * exp(0.09613020351321211 * x)


In [8]:
# Denver Metropolitan Area
# https://en.wikipedia.org/wiki/Denver_metropolitan_area
denver_metropolitan = dict(
    Colorado=[
        "Denver",
        "Arapahoe",
        "Jefferson",
        "Adams",
        "Douglas",
        "Broomfield",
        "Elbert",
        "Park",
        "Clear Creek",
        "Gilpin",
    ],
)

plot_states_and_counties_curve(
    df, denver_metropolitan, "Cases for the Denver Metropolitan Area"
)

y ≈ exp(4.93297330757162) * exp(0.08670815990334647 * x)
y = 138.7915694744893 * exp(0.08670815990334647 * x)
